In [ ]:
import sys
sys.path.append('..')
from span_pair2vec.embeddings.play import Play
# from embeddings.play import Play
# import numpy as np
# from heapq import nlargest
# from sklearn.metrics.pairwise import cosine_similarity

# from my_util import ntopidx
# from vdbscan import do_cluster

# d = Play('data/result_3/best_model.pt', 'data/result_3/saved_config.json', 'data/rel_20.txt')
# keywords = open('data/keyword_p.txt').read().strip().split('\n')
# keywords_token, keywords = d.token_align(keywords, 6)
# relations = open('data/rel_20.txt').read().strip().split('\n')
# relations_token = [rel.split() for rel in relations]
# relations = [(rel[:rel.index('<pad>')].strip() if '<pad>' in rel else rel) for rel in relations]

In [ ]:
# Helper functions for finding similar keywords and relations
def read_test_file(file_name:str):
    ret = []
    with open(file_name) as f_in:
        for line in f_in:
            central_kw, kws = line.strip().split(':')
            ret.append((central_kw, kws.split(',')))
    return ret

def run_test(test_data:list, d:Play, keywords:list, keywords_token:list, relations:list):
    ret = []
    for central_kw, kws in test_data:
        central_kw_token = central_kw.split()
        kws_token, kws = d.token_align(kws, 6)
        general_rel_prediction = d.get_prediction(keywords_token, [central_kw_token] * len(keywords_token))
        test_rel_prediction = d.get_prediction(kws_token, [central_kw_token] * len(kws_token))
        rel_score = cosine_similarity(test_rel_prediction, d.relation_representation)
        rel_predict_score = cosine_similarity(test_rel_prediction, general_rel_prediction)
        for i in range(len(kws)):
            rel_top_40 = [relations[idx] for idx in ntopidx(40, rel_score[i])]
            kws_top_40 = [keywords[idx] for idx in ntopidx(80, rel_predict_score[i])]
            ret.append((kws[i], central_kw, rel_top_40, kws_top_40))
    return ret

def write_result(data:list, file_name):
    with open(file_name, 'w', encoding='utf-8') as f_out:
        content = []
        for kw, central_kw, similar_rels, similar_kws in data:
            content.append('!%s<=>%s\n' % (kw, central_kw))
            content.append('>Similar Relation')
            content += similar_rels
            content.append('\n>Similar Keyword')
            content += similar_kws
            content.append('\n')
        f_out.write('\n'.join(content))

def do_test(test_file:str, result_file:str, d:Play, keywords:list, keywords_token:list, relations:list):
    test_data = read_test_file(test_file)
    result = run_test(test_data, d, keywords, keywords_token, relations)
    write_result(result, result_file)

In [ ]:
# Helper functions for doing relation clustering
def find_rel(target_rel:str, rel_list:list) -> list:
    ret = []
    for i, line in enumerate(rel_list):
        if target_rel in line:
            ret.append((line, i))
    return ret

def find_group_member(target:int, clusters:dict):
    for value in clusters.values():
        if target in value:
            return value

def find_most_similar(target:int, vecs:np.ndarray, relation_representation:np.ndarray, n:int=10):
    target_vec = relation_representation[vecs]
    similarities = cosine_similarity(target_vec, vecs)
    return ntopidx(n, similarities)

In [ ]:
do_test('test/non_overlap_test.txt', 'result/non_overlap_out_2.txt', d, keywords, keywords_token, relations)

In [ ]:
# Generate clusters
k = 7
cluster_id = do_cluster(d.relation_representation, k)
group_num = max(cluster_id) + 1
rel_id = np.arange(len(cluster_id))
rel_clusters = {}
for cid in range(-1, group_num):
    rel_clusters[cid] = set(rel_id[cluster_id == cid])

In [ ]:
from sklearn.cluster import DBSCAN
temp = DBSCAN(eps=0.1 , min_samples=5, metric='cosine').fit(d.relation_representation)
with open('result/relation_cluster.txt', 'w', encoding='utf-8') as f_out:
    content = []
    for i in range(max(temp.labels_)):
        content.append(str(i))
        s = np.arange(len(temp.labels_))[temp.labels_ == i]
        content += [relations[i] for i in s]
        content.append('')
    f_out.write('\n'.join(content))

In [ ]:
from my_keywords import Keyword_Base

kb = Keyword_Base()
kb.load_word_tree('data/wordtree.json')

In [ ]:
def process_triple(wordtree:dict, line:str):
    if not line:
        return False
    items = line.split(';')
    if len(items) != 3:
        return False
    if len(items[1].split()) > 6:
        return False
    for phrase in (items[0], items[2]):
        word_tokens = phrase.split()
        if len(word_tokens) > 6:
            return False
        i = 0
        keyword_num = 0
        has_key = False
        while i < len(word_tokens):
            if word_tokens[i] not in wordtree.keys():
                i += 1
            else:
                # If the word is the start of a key word
                it = wordtree
                while i < len(word_tokens) and word_tokens[i] in it.keys():
                    if "" in it[word_tokens[i]].keys():
                        # If the word could be the last word of a keyword, the pharse fulfills requirement
                        has_key = True
                        break
                    # Go down the tree to the next child
                    it = it[word_tokens[i]]
                    i += 1
            if has_key:
                break
        if not has_key:
            return False
    return True

In [ ]:
with open('data/ollie_triple.txt') as f_in:
    content = []
    for line in f_in:
        line = line.lower().strip()
        if process_triple(kb.MyTree, line):
            content.append(line)
    with open('data/ollie_triple_f.txt', 'w', encoding='utf-8') as f_out:
        f_out.write('\n'.join(content))

In [ ]:
# Helper functions for finding similar keywords
def find_similar_keywords(test_data:list, d:Play, keywords:list, keywords_token:list):
    ret = []
    kws_token, kws = d.token_align(test_data, 6)
    general_keywords_representation = d.encode_keyword_phrase(keywords_token)
    test_keywords_representation = d.encode_keyword_phrase(kws_token)
    score = cosine_similarity(test_keywords_representation, general_keywords_representation)
    for i in range(len(kws)):
        kws_top_40 = [keywords[idx] for idx in ntopidx(40, score[i])]
        ret.append((kws[i], kws_top_40))
    return ret

In [ ]:
with open('result/similar_keywords.txt', 'w', encoding='utf-8') as f_out:
    test_data = ['binary search tree', 'linked list', 'hash table', 'natural language processing', 'computer vision', 'speech recognition', 'binary search algorithm', 'linear regression', 'logistic regression', 'machine learning']
    similar_keywords = find_similar_keywords(test_data, d, keywords, keywords_token)
    content = []
    for kw, similar_kws in similar_keywords:
        content.append(kw)
        content.append('')
        content += similar_kws
        content.append('')
    f_out.write('\n'.join(content))